# Tutorial

## Basic imports and install library if needed

Install our library if needed by running the following cell.

```bash
pip install peerannot
```

In [ ]:
import numpy as np
from pathlib import Path

In [ ]:
DIR = Path().cwd()
DIRc10h = (DIR / ".." / "datasets" / "cifar10H").resolve()
DIR_module = DIRc10h / "cifar10h.py"
print(DIRc10h)

## Install dataset

In [ ]:
! peerannot install -h

In [ ]:
! peerannot install $DIR_module

# Aggregate labels with majority voting

Let us consider the majority vote system:
$$\hat y_i = \arg\max_{k} \sum_{j:j \text{ answered }i} 1\!\!1\{y_{i}^{(j)}=k\}$$

In [ ]:
! peerannot aggregate -h

In [ ]:
! peerannot aggregate $DIRc10h -s MV

## Dataset API loader

In [ ]:
from peerannot.runners.train import load_all_data
labels_path = DIRc10h / "labels" / "labels_cifar-10h_mv.npy"
trainset, valset, testset = load_all_data(
    DIRc10h, labels_path, path_remove=None, labels=labels_path, img_size=32, data_augmentation=False)

In [ ]:
trainset

## Dataset CLI load and train

In [ ]:
! peerannot train -h

In [ ]:
labels_path = DIRc10h / "labels" / "labels_cifar-10h_mv.npy"
num_epochs = 10

In [ ]:
! peerannot train $DIRc10h -o cifar10H_MV -K 10\
    --labels $labels_path --model resnet18 --img-size=32\
    --n-epochs=$num_epochs --lr=0.1 --scheduler -m 50 -m 100 \
    --num-workers 8

# Aggregate into soft labels with Dawid and Skene

In [ ]:
! peerannot aggregate $DIRc10h -s DS

In [ ]:
labels_path = DIRc10h / "labels" / "labels_cifar-10h_ds.npy"
num_epochs = 10

In [ ]:
! peerannot train $DIRc10h -o cifar10H_DS -K 10\
    --labels $labels_path --model resnet18 --img-size=32\
    --n-epochs=$num_epochs --lr=0.1 --scheduler -m 50 -m 100 \
    --num-workers 8

# Task ambiguity identification

In [ ]:
! peerannot identificationinfo

In [ ]:
! peerannot identify -h

In [ ]:
path_votes = DIRc10h / "answers.json"

In [ ]:
! peerannot identify $DIRc10h -K 10 --method WAUMstacked --labels $path_votes\
    --model resnet18 --n-epochs 10 --lr=0.1 --img-size=32 \
    --maxiter-DS=50